In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os

# 参数定义
part1_defect_rate = [0.1, 0.2, 0.1, 0.2, 0.1, 0.05]
part2_defect_rate = [0.1, 0.2, 0.1, 0.2, 0.2, 0.05]
final_product_defect_rate = [0.1, 0.2, 0.1, 0.2, 0.1, 0.05]

part1_cost = 4
part2_cost = 18
assemble_cost = 6
detect_cost_part1 = [2, 2, 2, 1, 8, 2]
detect_cost_part2 = [3, 3, 3, 1, 1, 3]
detect_cost_final = [3, 3, 3, 2, 2, 3]
market_price = 56
replace_loss = [6, 6, 30, 30, 10, 10]
decompose_cost = [5, 5, 5, 5, 5, 40]

# 生产成品数量
N = 100
# 蒙特卡洛模拟次数
simulations = 100

# 计算多个成品的销量与利润
def compute_total_sales_and_profit(d1, d2, d3, d4, N, situation):
    defect_rate_part1 = part1_defect_rate[situation] if d1 == 0 else 0
    defect_rate_part2 = part2_defect_rate[situation] if d2 == 0 else 0
    basic_defect_rate = final_product_defect_rate[situation]
    product_defect_rate = 1 - (1 - defect_rate_part1) * (1 - defect_rate_part2) * (1 - basic_defect_rate)
    
    # 计算合格产品数量与销售额
    total_sales = N * (1 - product_defect_rate)
    total_revenue = total_sales * market_price

    # 计算成本
    part1_total_cost = N * (part1_cost + (detect_cost_part1[situation] if d1 == 1 else 0))
    part2_total_cost = N * (part2_cost + (detect_cost_part2[situation] if d2 == 1 else 0))
    assembly_total_cost = N * (assemble_cost + (detect_cost_final[situation] if d3 == 1 else 0))
    
    # 如果未检测成品且次品，则产生调换损失
    replacement_loss = N * product_defect_rate * replace_loss[situation] if d4 == 0 else 0
    
    # 拆解成本
    decompose_total_cost = decompose_cost[situation] if d4 == 1 else 0

    # 总成本
    total_cost = part1_total_cost + part2_total_cost + assembly_total_cost + replacement_loss + decompose_total_cost
    
    # 总利润 = 总收入 - 总成本
    total_profit = total_revenue - total_cost
    
    return total_sales, total_profit

# 遗传算法的选择、交叉和变异操作
def genetic_algorithm_optimization(simulations, generations=10, population_size=10, mutation_rate=0.1):
    population = [(np.random.randint(0, 2), np.random.randint(0, 2), 
                   np.random.randint(0, 2), np.random.randint(0, 2)) for _ in range(population_size)]
    
    best_strategies = []
    best_sales_per_generation = []
    best_profit_per_generation = []
    avg_profit_per_generation = []
    
    for generation in range(generations):
        sales_for_population = []
        total_profit_generation = []
        
        for strategy in population:
            d1, d2, d3, d4 = strategy
            total_sales_list = []
            total_profit_list = []
            for _ in range(simulations):
                situation = np.random.randint(0, len(part1_defect_rate))
                total_sales, total_profit = compute_total_sales_and_profit(d1, d2, d3, d4, N, situation)
                total_sales_list.append(total_sales)
                total_profit_list.append(total_profit)
            avg_sales = np.mean(total_sales_list)
            avg_profit = np.mean(total_profit_list)
            sales_for_population.append((strategy, avg_sales, avg_profit))
            total_profit_generation.append(avg_profit)
        
        best_strategy, best_sales, best_profit = max(sales_for_population, key=lambda x: x[2])
        best_strategies.append(best_strategy)
        best_sales_per_generation.append(best_sales)
        best_profit_per_generation.append(best_profit)
        avg_profit_per_generation.append(np.mean(total_profit_generation))
        
        sorted_population = sorted(sales_for_population, key=lambda x: x[2], reverse=True)
        parent_population = [x[0] for x in sorted_population[:population_size // 2]]
        
        new_population = []
        for _ in range(population_size // 2):
            parent1_idx, parent2_idx = np.random.choice(len(parent_population), size=2, replace=False)
            parent1 = parent_population[parent1_idx]
            parent2 = parent_population[parent2_idx]
            
            crossover_point = np.random.randint(1, 4)
            offspring1 = parent1[:crossover_point] + parent2[crossover_point:]
            offspring2 = parent2[:crossover_point] + parent1[crossover_point:]
            new_population.extend([offspring1, offspring2])
        
        for i in range(len(new_population)):
            if np.random.rand() < mutation_rate:
                mutation_idx = np.random.randint(0, 4)
                new_population[i] = list(new_population[i])
                new_population[i][mutation_idx] = 1 - new_population[i][mutation_idx]
                new_population[i] = tuple(new_population[i])
        
        population = new_population

    return best_strategies, best_sales_per_generation, best_profit_per_generation, avg_profit_per_generation

# 定义生成结果的文件夹
output_dir = "output_generations"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 保存图表和结果的函数
def save_results_and_plots(generations, best_sales_per_generation, avg_profit_per_generation, best_strategy_final, best_sales_final, best_profit_final, avg_profit_final):
    # 保存每代最优期望销量图
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, generations + 1), best_sales_per_generation, marker='o', linestyle='-', color='b', label='Best Sales')
    plt.title(f'Best Expected Sales Over Generations (Generations={generations})')
    plt.xlabel('Generation')
    plt.ylabel('Best Expected Sales')
    plt.grid(True)
    best_sales_plot_file = os.path.join(output_dir, f'best_sales_{generations}.png')
    plt.savefig(best_sales_plot_file)
    plt.close()

    # 保存每代平均期望利润图
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, generations + 1), avg_profit_per_generation, marker='x', linestyle='--', color='g', label='Average Profit')
    plt.title(f'Average Profit Over Generations (Generations={generations})')
    plt.xlabel('Generation')
    plt.ylabel('Average Profit (Yuan)')
    plt.grid(True)
    avg_profit_plot_file = os.path.join(output_dir, f'avg_profit_{generations}.png')
    plt.savefig(avg_profit_plot_file)
    plt.close()

    # 保存文本结果
    result_file = os.path.join(output_dir, f'result_{generations}.txt')
    with open(result_file, 'w', encoding='utf-8') as f:
        f.write(f"Generations={generations}\n")
        f.write(f"最优策略: {best_strategy_final}\n")
        f.write(f"期望销量: {best_sales_final:.2f}\n")
        f.write(f"最佳利润: {best_profit_final:.2f} 元\n")
        f.write(f"平均利润: {avg_profit_final:.2f} 元\n")

# 生成范围为100到500，每次间隔50的 generations
generation_range = range(100, 501, 50)
population_size = 10
results = {}

for generations in generation_range:
    best_strategies, best_sales_per_generation, best_profit_per_generation, avg_profit_per_generation = genetic_algorithm_optimization(simulations, generations, population_size)

    # 获取最终结果
    best_strategy_final = best_strategies[-1]
    best_sales_final = best_sales_per_generation[-1]
    best_profit_final = best_profit_per_generation[-1]
    avg_profit_final = avg_profit_per_generation[-1]

    # 保存每次结果和图
    save_results_and_plots(generations, best_sales_per_generation, avg_profit_per_generation, best_strategy_final, best_sales_final, best_profit_final, avg_profit_final)

    # 保存结果到字典（如果需要进一步处理）
    results[generations] = {
        'best_strategy': best_strategy_final,
        'best_sales': best_sales_final,
        'best_profit': best_profit_final,
        'avg_profit': avg_profit_final
    }

print(f"All results and plots have been saved to the directory: {output_dir}")
